<a href="https://colab.research.google.com/github/LeeTunMT/python/blob/main/Main_3_old_2phase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROJECT LINEAR PROGRAMMING SOLVER - MTP TEAM**

## Introduction

The Linear Programming Solver was designed to solve linear optimization problems efficiently. It supports various solving methods, making it versatile for different types of linear programming (LP) challenges.

## Features

1. **User-Friendly Input**: Easily specify objective functions, constraints, and variable bounds.
2. **Multiple Methods**:
    - **Simplex Method**: Classic approach for solving LP problems.
    - **Two-Phase Simplex**: Finds feasible solutions with artificial variables.
    - **Dual Simplex**: Efficient for initially infeasible problems.
    - **Bland's Rule**: Prevents cycling in the Simplex method.
3. **Automatic Normalization**: Converts constraints and objectives to standard form.
4. **Detailed Results**: Provides optimal solutions, objective values, and solution steps.
5. **Error Detection**: Identifies infeasibility, unboundedness, and cycling.

## Usage

Define the problem interactively:
- Specify the objective function (minimize/maximize).
- Input constraints and their bounds.
- Define variable bounds.

The solver normalizes the problem, selects the appropriate method, and provides the solution with detailed output.

## Import Library

In [156]:
import numpy as np
import pandas as pd
import warnings

In [157]:
warnings.filterwarnings("ignore")

# INPUT

# Objective function

In [158]:
def Goal_func(MinMax, variables):
    arrayObj_func = {}                                                     # Khởi tạo arrayObj_func , type = dict{}. Mục đích là để lưu các hệ số của từng biến
    print("HÀM MỤC TIÊU")                                                                   # trong hàm mục tiêu
    print("Nhập các hệ số của hàm mục tiêu: ")
    for i in range(1, variables + 1):                                      # vòng lặp để nhập hệ số của các biến trong hàm mục tiêu
        arrayObj_func['x' + str(i)] = float(input('\tHệ số của x' + str(i) + ' của hàm mục tiêu: '))   # lưu các hệ số đó vào arrayObj_func, với mỗi xi tương ứng. vđ {'x1': 1, 'x2': 3}

    #Tiếp theo là in ra hàm mục tiêu
    print("Hàm mục tiêu:   ", end='')
    if MinMax == 0:
        print('Min ', end='')
    else:
        print('Max ', end='')
    for i in range(1, variables + 1):
        if i != 1 and arrayObj_func['x' + str(i)] > 0:                     # Nếu hệ số không phải là của x1 và hệ số dương thì in thêm dấu '+' trước hệ số
            print(' + ', end='')
        else:
            print(' ', end='')                                             # Nếu hệ số là của x1 và hệ số âm thì chỉ cần thêm vào 1 khoảng trống trước hệ số
        print(str(arrayObj_func['x' + str(i)]) + '*x' + str(i), end='')    # In ra giá trị của hệ số

    return arrayObj_func

## Constraints

In [159]:
def constraints(variables, quantity):
    arrayQuantity = {}                                                    # Khởi tạo arrayObj_func , type = dict{}. Mục đích là để lưu các hệ số của từng biến
    print("\n\nCÁC RÀNG BUỘC BIẾN")                                                                  # trong các ràng buộc
    for i in range(1, variables + 1):
        arrayQuantity['x' + str(i)] = [0 for j in range(quantity)]        # Cách làm tương tự như hàm mục tiêu, nhưng đầu tiên sẽ khởi tạo các hệ số là 0 trước

    arrayQuantity['limited'] = [0 for j in range(quantity)]               # limited sẽ lưu các ràng buộc ≤, = và ≥
    arrayQuantity['HeSo'] = [0 for j in range(quantity)]                  # HeSo sẽ lưu các giới hạn của ràng buộc (tức bi trong QHTT)

    print("Nhập các hệ số của các ràng buộc và giới hạn của nó: ")
    for i in range(quantity):                                             #Khởi tạo vòng lặp để nhập các giá trị cần thiết cho ràng buộc
        print("Ràng buộc thứ " + str(i + 1))
        for j in range(1, variables + 1):                                 #Khởi tạo vòng lặp để nhập từng hệ số của biến trong từng ràng buộc
            arrayQuantity['x' + str(j)][i] = float(input('\tHệ số của x' + str(j) + ' của ràng buộc thứ ' + str(i+1) + ': '))       #Nhập hệ số từng biến

        arrayQuantity['limited'][i] = int(input('\tNhập vào ràng buộc ( ≤ : 0, ≥ : 1, = : 2): '))    #nhập vào ràng buộc ≤, = và ≥
        arrayQuantity['HeSo'][i] = int(input('\tNhập vào giới hạn của ràng buộc: '))          #nhập vào giới hạn của ràng buộc

    return arrayQuantity

In [160]:
def show_contraints(arrayQuantity, quantity):
    #In ra các ràng buộc để người dùng kiểm tra lại
    print("Các ràng buộc biến vừa nhập: ")
    for i in range(quantity):
        for j in arrayQuantity:
            if j != 'x1' and j != 'limited' and j != 'HeSo' and arrayQuantity[j][i] > 0:
                print('  +', end='')
                print(str(arrayQuantity[j][i]) + '*' + str(j), end='')
            elif j == 'limited':
                if arrayQuantity[j][i] == 0:
                    arrayQuantity[j][i] = '≤'
                elif arrayQuantity[j][i] == 1:
                    arrayQuantity[j][i] = '≥'
                elif arrayQuantity[j][i] == 2:
                    arrayQuantity[j][i] = '='
                print(" ",arrayQuantity[j][i], end='')
            elif j == 'HeSo':
                if arrayQuantity[j][i] > 0:
                    print(end=' ')
                print(' ', str(arrayQuantity[j][i]), end='')
            else:
                if j == 'x1' and arrayQuantity[j][i] > 0:
                    print(end=' ')
                print(' ',str(arrayQuantity[j][i]) + '*' + str(j), end='')
        print()

### Constraints of each variable, ≤ 0, ≥ 0 or freedom

In [161]:
def constraint_for_variables(variables):
    #Nhập vào ràng buộc của từng biến
    print("\nRÀNG BUỘC DẤU")
    print("Nhập ràng buộc dấu của từng biến: ")
    arrayVariables = {}                                                           #Khởi tạo arrayVariables type dict{}, lưu các ràng buộc của từng biến
    for i in range(1, variables + 1):                                             #dict sẽ có dạng ví dụ như sau: arrayVariables = {'x1': 0, 'x2': 1}
                                                                            # Tương ứng với các giá trị 0, 1, 2 sẽ là ≤ 0, ≥ 0, tự do
        arrayVariables['x' + str(i)] = int(input('\tBiến ' + 'x ' + str(i) + ' (x ≤ 0 : 0;  x ≥ 0 : 1; biến tự do : 2) :'))

    print('Các ràng buộc của từng biến là:')
    for i in arrayVariables:
        if arrayVariables[i] == 0:
            print('\t' + i + ' ≤ 0')
        elif arrayVariables[i] == 1:
            print('\t' + i + ' ≥ 0')
        else:
            print('\t' + i + ' tự do')

    return arrayVariables

In [162]:
def  Initialize_the_problem():
    MinMax = int(input("NHẬP BÀI TOÁN QUY HOẠCH TUYẾN TÍNH".center(120) + "\nNhập min/max của hàm mục tiêu (0 là min, 1 là max): "))
    variables = int(input("Nhập số lượng biến: "))
    quantity = int(input("Nhập số lượng ràng buộc của biến: "))

    print('NHẬP CÁC HỆ SỐ VÀ DẤU RÀNG BUỘC CHO HÀM MỤC TIÊU VÀ CÁC RÀNG BUỘC BIẾN'.center(120))
    arrayObj_func =  Goal_func(MinMax, variables)
    arrayQuantity = constraints(variables, quantity)
    show_contraints(arrayQuantity, quantity)
    arrayVariables = constraint_for_variables(variables)

    return MinMax, variables, quantity, arrayObj_func, arrayQuantity, arrayVariables

## Standardization

##### Standardize variable constraints

In [164]:
def Normalize_constraint_variable(arrayVariables, arrayQuantity, log_record, arrayObj_func, variables, quantity):
    for i in arrayVariables:
        # Chuẩn hóa khi ràng buộc về biến là ≤ 0
        if arrayVariables[i] == 0:
            log_record[i] = '-' + i  # Thay đổi biến đó thành dạng VD x1 = -x1
            arrayObj_func[i] = -arrayObj_func[i]  # Sửa hàm mục tiêu
            for j in range(quantity):  # Sửa các ràng buộc
                arrayQuantity[i][j] = -arrayQuantity[i][j]

        # Chuẩn hóa khi ràng buộc về biến là tự do
        if arrayVariables[i] == 2:
            new_var = 'x' + str(variables + 1)
            log_record[i] = [i, new_var]  # Thêm biến mới
            arrayObj_func[new_var] = -arrayObj_func[i]  # Sửa hàm mục tiêu cho biến mới
            arrayQuantity[new_var] = [-arrayQuantity[i][j] for j in range(quantity)]  # Sửa ràng buộc cho biến mới
            variables += 1

    return arrayObj_func, variables, log_record, arrayQuantity


##### Standardize constraints

In [165]:
def Normalize_constraints(arrayQuantity, quantity):
    for j in range(quantity):
        if arrayQuantity['limited'][j] == '≥':  # Với các ràng buộc ≥
            for i in arrayQuantity:
                if i != 'limited':
                    arrayQuantity[i][j] = -arrayQuantity[i][j]
            arrayQuantity['limited'][j] = '≤'  # Thay đổi ràng buộc thành ≤

        if arrayQuantity['limited'][j] == '=':  # Với các ràng buộc =
            for i in arrayQuantity:
                if i != 'limited':
                    arrayQuantity[i].append(arrayQuantity[i][j])
                    arrayQuantity[i][j] = -arrayQuantity[i][j]
            arrayQuantity['limited'].append('≤')
            arrayQuantity['limited'][j] = '≤'
            quantity += 1

    return arrayQuantity, quantity

##### Standardize goal function

In [166]:
def Normalize_GoalFunction(MinMax, arrayObj_func):
    if MinMax == 1:  # Chuẩn hóa hàm mục tiêu nếu là max
        for i in arrayObj_func:
            arrayObj_func[i] = -arrayObj_func[i]
    return arrayObj_func

In [167]:
def Normalize(MinMax, arrayObj_func, arrayQuantity, arrayVariables, quantity, variables):
    # Lưu lại các thay đổi về biến khi chuẩn hóa
    log_record = {}

    # Chuẩn hóa ràng buộc biến
    arrayObj_func, variables, log_record, arrayQuantity = Normalize_constraint_variable(arrayVariables, arrayQuantity, log_record, arrayObj_func, variables, quantity)

    # Chuẩn hóa các ràng buộc
    arrayQuantity, quantity = Normalize_constraints(arrayQuantity, quantity)

    # Chuẩn hóa hàm mục tiêu
    arrayObj_func = Normalize_GoalFunction(MinMax, arrayObj_func)

    return arrayObj_func, variables, log_record, arrayQuantity, quantity

## Put data into dataframe

In [ ]:
def Initialize_the_problem_into_table(variables, quantity, arrayQuantity, arrayObj_func):
    #### tạo một list chuẩn bị
    #Bước này để lấy các hệ số trong ràng buộc trước đồng thời khởi tạo dataframe để giải bài toán dưới dạng bảng trong QHTT
    prepare = {}
    for i in range(1, variables + 1):
        prepare['x' + str(i)] = arrayQuantity['x' + str(i)]
        temp = arrayQuantity['x' + str(i)]
    df = pd.DataFrame(prepare)

    #Hàm này để đưa hàm mục tiêu vào dataframe
    Obj_func_row = pd.DataFrame(arrayObj_func, index=[0])
    df = pd.concat([Obj_func_row, df.loc[:]]).reset_index(drop=True)
    # Sau đó là thêm vào các w0,w1,w2 và giới hạn của từng ràng buộc
    HeSo = [0]
    for i in range(1,quantity + 1):
        df['w' + str(i)] = [0 for j in range(quantity + 1)]
        df['w' + str(i)][i] = 1
        HeSo.append(arrayQuantity['HeSo'][i-1])
    df['HeSo'] = HeSo
    return df

## Access to each math problem

#### Dantzig function

In [ ]:
def dantzig(array):
        array = np.array(array, dtype=np.float64) #dataframe thành array
        pivot_row = -1 # khởi tạo biến, dùng để sử dụng theo dõi hàng pivot
        pivot_col = -1 # và cột pivot
        step_count = 0  # Biến đếm bước xoay từ vựng
        while array[0, :-1].min() < 0: # duyệt giá trị hàm mục tiêu z có giá trị nhỏ nhất âm hay không, nếu có thì tiếp tục vòng lặp
            pivot_col = np.argmin(array[0, :-1]) # gán vị trí của giá trị nhỏ nhất trong hàm mục tiêu vào pivot_col tìm biến vào
            if (array[1:, pivot_col] <= 0).all(): # tìm biến ra, nếu các giá trị trong cột tương ứng với pivot_col(trừ hàng đầu tiên) nhỏ hơn hoặc bằng 0
                return -1, array, step_count # trả về -1 ( ứng với trường hợp không giới nội)
            else:
                ratios = array[1:, -1] / array[1:, pivot_col] # lấy hệ số b tương ứng với cột (array[1:-1]) chia cho giá trị trong cột pivot_col để tìm biến ra.
                ratios[array[1:, pivot_col] <= 0] = np.inf # gán giá trị ratios âm hoặc nhỏ hơn 0 vào giá trị inf để loại trừ nó và khỏi xem xét
                pivot_row = np.argmin(ratios) + 1 # vị trí dòng có hệ số ratios nhỏ nhất(+1 vì bỏ dòng đầu tiên )
            array[pivot_row, :] /= array[pivot_row, pivot_col] # biến đổi để giá trị ở array[pivot_row, pivot_col] = 1, và giá trị khác trong pivot_col = 0.
            #pivot column lúc này trở thành biến cơ sở
            for i in range(len(array)): # biến đổi các biến lại
                if i != pivot_row:
                    array[i, :] -= array[i, pivot_col] * array[pivot_row, :]
            step_count += 1  # Tăng bước xoay từ vựng sau mỗi vòng lặp
            #display(pd.DataFrame(array))
        return 1, array, step_count


#### Bland function

In [ ]:
#Hàm Bland tương tự như hàm Dantzig
def Bland(array):
    array = np.array(array, dtype=np.float64)
    pivot_row = -1
    pivot_col = -1
    step_count = 0  # Biến đếm bước xoay từ vựng
    while array[0, :-1].min() < 0:
        # gán vị trí âm có chỉ số nhỏ nhất( x1 > x2 > ...> w1 > w2> ....)
        pivot_col = np.where(array[0, :-1] < 0)[0][0]
        if (array[1:, pivot_col] <= 0).all():
            return -1, array, step_count
        else:
            ratios = array[1:, -1] / array[1:, pivot_col]
            ratios[array[1:, pivot_col] <= 0] = np.inf
            pivot_row = np.argmin(ratios) + 1
        array[pivot_row, :] /= array[pivot_row, pivot_col]
        for i in range(len(array)):
            if i != pivot_row:
                array[i, :] -= array[i, pivot_col] * array[pivot_row, :]
        step_count += 1  # Tăng bước xoay từ vựng sau mỗi vòng lặp
    return 1, array, step_count

#### Two Phase function

In [ ]:
# Ham xay dung bai toan bo tro dua tren bai toan goc
def SupplementaryMath(data, cols_data, rows_data):
    df = data.copy()
    # Cho he so cua cac phan tu o ham muc tieu bang 0
    for i in range(0, cols_data):
        df.iloc[[0], [i]] = 0

    # Tao cot x0
    x0_values = [1] # Cho he so cua cac phan tu dau tien bang 1
    for i in range(1, rows_data): # Cho he so cua cac phan tu con lai bang -1
        x0_values.append(-1)

    # Them cot x0 vao data frame
    df.insert(cols_data-1, 'x0', x0_values)

    return df

##### Phase 1

In [ ]:
# Ham tim vi tri cua phan tu x0
def Find_x0_Index(df):
    features_name = list(df.columns)
    x0_index = features_name.index('x0')
    return x0_index

In [ ]:
# Ham chuyen -0.0 -> +0.0
def ConvertNegativeZero(matrix):
    matrix += 0.0

In [ ]:
# Ham tim phan tu cua bien vao
def FindInputIndex(array, cols, rows):

    min = array[0]
    min_index = 0
    for i in range(1, cols):
        if array[i] < min:
            min = array[i]
            min_index = i

    return min_index

In [ ]:
def FirstStepOfPhase1(matrix, cols, rows, x0_index):

    min = matrix[1][-1]/abs(matrix[1][x0_index])
    row_min_index = 1
    for i in range(2, rows):
        if matrix[i][-1]/abs(matrix[i][x0_index]) < min:
            min = matrix[i][-1]/abs(matrix[i][x0_index])
            row_min_index = i

    for i in range(0, rows):
        if i != row_min_index:
            coef = matrix[i][x0_index]/matrix[row_min_index][x0_index]
            matrix[i][:] -= coef * matrix[row_min_index][:]
    matrix[row_min_index][:] /= matrix[row_min_index][x0_index]

    ConvertNegativeZero(matrix)

    return matrix

In [ ]:
def RemainingSteps(matrix, cols, rows, input_index):

    min = 100000000
    row_min_index = -1
    for i in range(1, rows):
        if matrix[i][input_index] > 0:
            if matrix[i][-1]/(matrix[i][input_index]) < min:
                min = matrix[i][-1]/(matrix[i][input_index])
                row_min_index = i
    for i in range(0, rows):
        if i != row_min_index and matrix[i][input_index] != 0:
            coef = matrix[i][input_index]/matrix[row_min_index][input_index]
            matrix[i][:] -= coef * matrix[row_min_index][:]
    matrix[row_min_index][:] /= matrix[row_min_index][input_index]

    ConvertNegativeZero(matrix)

    return matrix


In [ ]:
# Ham kiem tra cac dieu kien
def CheckTheConditions(df, matrix, cols, rows, x0_index):
    # Xoay lan dau tien doi voi x0 > 0
    matrix = FirstStepOfPhase1(matrix, cols, rows, x0_index)

    while True:
        # Neu cac phan tu khac deu co he so bang 0 va x0 quay tro lai la bien chua co so
        if np.all(matrix[0][:-2] == 0) == True and matrix[0][-1] == 0 and matrix[0][x0_index] == 1:
            check = 2
            break
        # Neu cac phan tu khac deu co he so lon hon hoac bang 0 va x0 khong quay tro lai la bien co so
        elif np.all(matrix[0][:-2] >= 0) == True and matrix[0][x0_index] == 0:
            check = 0
            break
        # Truong hop con lai tim bien vao co so, tiep tuc xoay
        else:
            epsilon = matrix[0][:-2]
            input_index = FindInputIndex(epsilon, cols-2, rows)
            matrix = RemainingSteps(matrix, cols, rows, input_index)

    return (matrix, check)

In [ ]:
# Ham tim nhung bien co so
def FindBasicVariables(matrix, cols, rows):
    # Tao danh sach chua vi tri cua bien co so trong cot va dong chua bien co so
    basic_variables_index = []

    for i in range(0, cols-1):
        c0 = matrix[:, i] == 0
        count0 = np.count_nonzero(c0 == True)
        c1 = matrix[:, i] == 1
        count1 = np.count_nonzero(c1 == True)

        # Bien co so la nhung bien ma cot cua no chua 1 gia tri 1 va cac gia tri con lai bang 0
        if count0 + count1 == rows:
            row_index = np.where(c1 == True)
            basic_variables_index.append((i, row_index[0][0]))

    return basic_variables_index

In [ ]:
# Ham tao ra z moi
def New_z(data, matrix_data, matrix, cols_data, rows_data, cols, rows):
    # Lay cac he so cua ham muc tieu o bai toan goc va bai toan bo tro
    z = matrix_data[0][:]
    z_new = matrix[0][:]

    # Tim cac bien co so sau khi giai bai toan bo tro
    basic_variables = FindBasicVariables(matrix, cols, rows)
    bv_index = [] # Tao danh sach chua vi tri cua cac bien co so trong moi cot
    bv_rows = [] # Tao danh sach chua dong chua bien co so trong mot cot
    # Them cac bien vi tri cua cac bien co so trong moi cot va dong cua no vao hai danh sach
    for i in range(0, len(basic_variables)):
        bv_index.append(basic_variables[i][0])
        bv_rows.append(basic_variables[i][1])
    dictionary_index_rows = dict(zip(bv_index, bv_rows))

    z_basic_variables_index = [] # Tao danh sach chua vi tri cua cac bien co so trong moi cot
    z_non_basic_variables_index = [] # Tao danh sach chua vi tri cua cac bien khong co so trong moi cot
    # Tien hanh them cac bien nay vao hai danh sach
    for i in range(0, cols_data-1):
        if i in bv_index and z[i] != 0: # Kiem tra neu bien nay la bien co so o z' va he so cua no o ham muc tieu goc khac 0
            z_basic_variables_index.append(i)
        elif i not in bv_index and z[i] != 0: # Kiem tra neu bien nay la bien khong co so o z' va he so cua no o ham muc tieu goc khac 0
            z_non_basic_variables_index.append(i)

    # Chuyen doi z ve z_new ung voi bai toan chap nhan duoc
    for i in range(0, cols-1):
        # Neu i dang xet la bien co so o bai toan bo tro
        if i in z_basic_variables_index:
            temp_i = dictionary_index_rows[i]
            temp = matrix[temp_i][:].copy()
            temp[i] = 0
            temp[0:] *= -1
            z_new += z[i] * temp
        # Neu i dang xet la bien khong co so o bai toan bo tro
        elif i in z_non_basic_variables_index:
            temp = np.zeros(cols_data)
            temp[i] = 1
            z_new += z[i] * temp

    return z_new

In [ ]:
# Ham tao ra bang chap nhan duoc
def AcceptableTable(data, matrix_data, df, matrix, cols_data, rows_data, cols, rows):
    # Tim z_new va them z_new vao ma tran
    z_new = New_z(data, matrix_data, matrix, cols_data, rows_data, cols, rows)
    matrix[0][:] = z_new

    ConvertNegativeZero(matrix)

    # Thay doi noi dung cho data frame goc
    features_name = list(df.columns)
    df = pd.DataFrame(matrix, columns=features_name)

    return df # Tra ve data frame chap nhan duoc

In [ ]:
# Ham thuc thi pha 1
def Phase1(data, matrix_data, df, matrix, cols_data, rows_data, cols, rows, x0_index):
    (matrix, check) = CheckTheConditions(df, matrix, cols, rows, x0_index)
    if check == 2:
        features_name = list(df.columns)
        df = pd.DataFrame(matrix, columns=features_name)
        df = df.drop(columns='x0')
        matrix = np.delete(matrix, np.s_[x0_index:x0_index+1], axis=1)

        df = AcceptableTable(data, matrix_data, df, matrix, cols_data, rows_data, cols, rows)
        return check, df
    return check, df

In [169]:
def two_Phase(data):
    # Lưu các thông số dòng và cột của data
    rows_data, cols_data = data.shape
    matrix_data = np.array(data, dtype=float)

    # Đưa về bài toán bổ trợ
    df = SupplementaryMath(data, cols_data, rows_data)

    # Lưu các thông số dòng và cột của df
    rows, cols = df.shape
    matrix = np.array(df, dtype=float)

    # Pha 1
    x0_index = Find_x0_Index(df)
    check, output = Phase1(data, matrix_data, df, matrix, cols_data, rows_data, cols, rows, x0_index)

    # Biến đếm bước xoay từ vựng
    step_count = 0

    # Nếu Pha 1 trả về check == 2
    if check == 2:
        while np.any(output[0, :-1] < 0):
            output = FirstStepOfPhase1(output, cols, rows, x0_index)
            step_count += 1
            if (output[0, :-1] < 0).all():
                check = -1
                break

    return check, output, step_count


#### Dual Simplex method

In [ ]:
def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def dual_simplex(array):
    array = np.array(array, dtype=np.float64)
    step_count = 0  # Counter for steps

    while np.any(array[1:, -1] < 0):  # While there is a negative value in the RHS
        pivot_row = np.argmin(array[1:, -1]) + 1  # Find the most negative value in RHS
        row = array[pivot_row, :-1]
        obj_func = array[0, :-1]

        ratios = []
        for i in range(len(row)):
            if row[i] < 0:
                ratios.append(obj_func[i] / row[i])
            else:
                ratios.append(np.inf)

        pivot_col = np.argmin(ratios)
        if ratios[pivot_col] == np.inf:
            return -1, array, step_count

        array = pivot(array, pivot_row, pivot_col)
        step_count += 1

    return 1, array, step_count

def pivot(array, pivot_row, pivot_col):
    array[pivot_row, :] /= array[pivot_row, pivot_col]
    for i in range(len(array)):
        if i != pivot_row:
            array[i, :] -= array[i, pivot_col] * array[pivot_row, :]
    return array

## Classify problem cases and apply appropriate solutions

In [ ]:
def is_basic(column):
    # Kiểm tra xem cột có phải là biến cơ sở không
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

In [ ]:
def Conclusion(check, output, MinMax, log_record, step_count, method):
    row, col = output.shape
    obj = 0
    x = 0

    print(f"Giải bằng phương pháp: {method}")

    if check == -1:
        print("Bài toán không giới nội.")
        obj = np.inf
        x = ("Không có nghiệm")
        print("Giá trị tối ưu của bài toán: ", obj)
        print(x)
    else:
        obj = output[0, -1]

        if MinMax == 1:
            obj = -obj
        print(output)

        if log_record != {}:
            for i in log_record:
                if log_record[i][0] != '-':
                    output = np.delete(output, int(log_record[i][1][1]) - 1, axis=1)
                    col -= 1

        for i in range(col - 1):
            if not is_basic(output[:, i]) and output[1, i] == 0:
                print("Bài toán vô số nghiệm.")
                x = ("Vô số nghiệm")
                break
            else:
                solutions = np.zeros(col - 1)
                for i in range(col - 1):
                    if is_basic(output[:, i]):
                        for j in range(1, row):
                            if output[j, i] == 1:
                                solutions[i] = output[j, -1]
                        x = solutions[:col - row]

                if log_record != {}:
                    x_new = []
                    for i in log_record:
                        if log_record[i][0] == '-':
                            x_new.append(0 - x[int(i[1]) - 1])
                    x = x_new

        print("Nghiệm tối ưu của bài toán: ")
        for i in range(len(x)):
            print('\tx' + str(i + 1) + ': ', x[i])
        print("Giá trị tối ưu của bài toán: ", -obj)
        print("Số bước xoay từ vựng: ", step_count)

## Main function

In [ ]:
def main():
    MinMax, variables, quantity, arrayObj_func, arrayQuantity, arrayVariables = Initialize_the_problem()
    ##### Chuẩn hóa bài toán
    arrayObj_func, variables, log_record, arrayQuantity, quantity = Normalize(MinMax, arrayObj_func, arrayQuantity, arrayVariables, quantity, variables)
    ### đua dữ liệu vào dataframe để giải bài toán theo dạng bảng
    df = Initialize_the_problem_into_table(variables, quantity, arrayQuantity, arrayObj_func)

    print('KẾT LUẬN'.center(100))
    if np.any(df['HeSo'][1:] < 0):  # Nếu trong dataframe có bất kỳ một giá trị bi nào nhỏ hơn 0 sẽ thực hiện 2 pha
        check, output, step_count = two_Phase(df)
        if check == 0:
            print('Bài toán vô nghiệm.')
        else:
            Conclusion(check, output, MinMax, log_record, step_count)

    elif np.all(df['HeSo'][1:] > 0):  # Nếu trong dataframe có tất cả giá trị bi lớn hơn 0 sẽ thực hiện dantzig
        array = []
        for i in range(np.shape(df)[0]):
            array.append([str(df.values[i, j]) for j in range(np.shape(df)[1])])
        check, output, step_count = dantzig(array)
        Conclusion(check, output, MinMax, log_record, step_count)

    elif np.all(df['HeSo'][1:] == 0):  # Nếu tất cả các hệ số của bi đều bằng 0, thực hiện phương pháp đối ngẫu
        array = []
        for i in range(np.shape(df)[0]):
            array.append([str(df.values[i, j]) for j in range(np.shape(df)[1])])
        check, output, step_count = dual_simplex(array)
        print("Giải bằng phương pháp đối ngẫu.")
        Conclusion(check, output, MinMax, log_record, step_count)

    else:  # Nếu không nằm trong 3 trường hợp trên thì thực hiện bland
        array = []
        for i in range(np.shape(df)[0]):
            array.append([str(df.values[i, j]) for j in range(np.shape(df)[1])])
        check, output, step_count = Bland(array)
        print("Giải bằng phương pháp Bland.")
        Conclusion(check, output, MinMax, log_record, step_count)

if __name__ == '__main__':
    main()

In [170]:
# Khởi tạo dữ liệu cho bài toán
data = pd.DataFrame({
    'x1': [1, 2, -3],
    'x2': [1, 1, -2],
    's1': [1, 0, 0],
    'RHS': [4, 5, 0]
})

# Gọi hàm two_Phase
check, output, step_count = two_Phase(data)

# Kiểm tra kết quả
print("Check:", check)
print("Output:")
print(output)
print("Step count:", step_count)

def is_solution_correct(output):
    optimal_value = output.iloc[0, -1]
    x1_value = output.iloc[1, -1]
    x2_value = output.iloc[2, -1]

    # Kiểm tra giá trị tối ưu
    if not np.isclose(optimal_value, 11.0):
        return False

    # Kiểm tra các nghiệm của biến
    if not (np.isclose(x1_value, 1.0) and np.isclose(x2_value, 3.0)):
        return False

    return True

# Kiểm tra độ chính xác của nghiệm
if check == 0:
    print("Bài toán vô nghiệm.")
elif check == -1:
    print("Bài toán không giới nội.")
else:
    if is_solution_correct(output):
        print("Nghiệm đúng.")
    else:
        print("Nghiệm sai.")


IndexError: index 0 is out of bounds for axis 0 with size 0